# Çoklu işlem

Peki her kod optimize edilebilir mi? Hayır.

Çoğu simülasyonlar zaten optimum durumda çalışmakta ve buna rağmen bir simülasyonun sonuç vermesi, süper bilgisayarlarla, haftalar, aylar alabilir. (Tabii bu simülasyonların evrenin ilk 12 milyar yılını simüle ediyor. Orası ayrı.)

Şimdi gelelim diğer yöntemlere. Varsayalım ki bir kod yazdınız ve olabildiğince optimizasyon yaptınız. Ama haka daha yüksek performansa ihtiyacınız var. Ne yapabiliriz?

Bunun için kodun yavaşlığının nedenini anlamak gerek. Koddali darboğazları (bottleneck) tespit edersek, farklı yaklaşımlarda bulunabiliriz.

Örneğin kodun sistem kaynaklarını nasıl kullandığını görmek gerek.

## Sistem kaynakları

Sistem kaynaklarından kasıt işlem gücü olan bilgisayar birimlerdir. Bu birimleri kontrol edersek, en çok zaman kaybını nerede yaşadığımızı anlar ve gerekli düzeltmeleri yapabiliriz.

Öncelikle bir örnek oluştualım. Diskte bulunan ```2000``` adet görüntü dosyasının (Toplamda ```2GB```) ortalamasını alalım.



### Disk

En gıcık sistem darbağazına neden olur. ```2000``` adet (```2GB```) dosyayı diskten okumamız gerek. Diskimiz saniyede 10MB okuma yapabiliyorsa:

In [1]:
2 * 1024 / 140

14.628571428571428

Diskten okuma işlemi ```10``` saniye sürecektir.

Böyle bir durumda, sistem monitörü yazılımlarından herhangi biriyle sistem kaynaklarınızı takip ederseniz, disk kullanımının ```10``` saniye boyuna ```%100``` olduğunu görürsünüz.

İyi leştirmek için çok fazla seçeneğiniz yoktur. 

:::{warning}
```2000``` adet dosyadan oluşan, ```2GB```'lık veriyi 10 adet ten oluşan ```2GB```lik veriye çevirebilrsek belki daha da hızlanabiliriz. Ama bu da ```20000``` dosyayı okumamız gerektiği anlamına gelir.
:::

Belki donanımı değiştirmek gerek. Bu nedenle sponsorumuz olan ```abuzittin donanım destek```'e teşekkür deriz. ```astrokod``` koduyla ```abuzittin donanım destek```'ten yapacağınız ilk alışverişte %3.5 indirim kazanabilirsiniz.

```{image} ../images/hahaha_2.gif
:alt: Python resmi web sayfası
:class: bg-primary mb-1
:width: 400px
:align: center
```

### Hafıza (RAM)

Belki de sisteminizin hafızası yetersizdir. Aslında bu durum genelde ```memoryError``` hatası alırsınız. Eğer optimizasyon yapmanıza rağmen (Örneğin tüm dosyaları hafızaya almaktansa, kümülatif biçimde toplarsınız) hafıza sorunu yaşıyorsanız, Donanımınızı güncellemeniz gerekebilir.

### İşlemci (CPU)

Bu kısım daha enterasandır. ```Python``` bir thread üzerinde çalışır. Yani işletim sistemi ```Python```'a bir bölge gösterir ve "git orda oyna" der.

Bu thread bir sanal çekirdek üzerinde çalışır. Kısacası sizin, bilgisayarınız ```8``` çekirdekli ise, sizin toplamda ```16``` sanal çekirdeğiniz var anlamına gelir. Bu ```16``` çekirdeğin ```1```'i size tahsis edilmiştir.

Bunu çözebilir miyiz? Evet...

```Python```'ın ```multiprocessing``` kütüphanesini kullanabilirsiniz.

Elinizdeki verinin farklı threadlerce işlenmesini isteyebilirsiniz. Bizim örneğimizde ```2000``` tane dosyanın RAM'de olduğunu varsayalım. Bunların toplanması gerek. En sonunda ise bütün değerleri ```2000```'e böler ortalamayı hesaplamış oluruz.

:::{note}
4 tane sanal çekirdek kullanacağımızı varsayalım.
:::

Benim size anlatabileceğim 2 multiprocess mantığı vardır.

1. Orak RAM ihtiyacı duyanlar
2. Ortak RAM ihtiyacı duymayanlar

#### Orak RAM ihtiyacı duyanlar
Bu durumda RAM üzerinde boş bir görüntü oluştururuz. Daha sonra bütün görüntüleri bu boş görüntüye ekleriz. En sonda ise bölme işlemini yaparız.



- RAM'de bir yerde boş görüntüyü oluşturdum
- 4 thread oluşturup, bu threadlere boş görüntünün yerini gösteririm.
- Toplayın arkadaşlar derim.

En sonunda ise bu toplam veriyi ```2000```'e bölelim.

#### Ortak RAM ihtiyacı duymayanlar
Bu durumda götüntülerimizi 4 tane ```500``` adet görüntüden oluşan gruplara böleriz. Her thread kısmetine düşen ```500``` görüntüyü toplar. En sonunda oluşan 4 görüntüyü de toplar ve ```2000```'e böleriz.

Burada threadler birbiriyle konuşmadı, aynı veri üzerinde çalışmadılar.